# 
<a id="data-prep"></a>
<div style="background-color: #000D5B; color: white; text-align: center; padding: 6px 0 22px 0">
    <h3 style="background-color: #000D5B; color: white; text-align: left">RMIT School of Computer Science and Technology</h3>
    <br/>
    <h1>COSC3007: Deep Learning</h1>
    <h2>Assignment 2: Stance Twitter Sentiment Analysis and Detection </h2>
    <h2> [2] MODELLING AND MODEL EVALUATIONS </h2>

</div>

# Import Libs

In [1]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import numpy as np

# [1] Load Data

In [2]:
train_df = pd.read_csv("./StanceDataset/processed_train.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])
test_df = pd.read_csv("./StanceDataset/processed_test.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])

# [2] Prepare label and fit data

## Split test and val

In [3]:
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

## Tweets

In [4]:

# Initialize and fit the tokenizer on the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['processed_tweet'])

# Convert texts to sequences for all datasets
train_sequences = tokenizer.texts_to_sequences(train_df['processed_tweet'])
test_sequences = tokenizer.texts_to_sequences(test_df['processed_tweet'])
val_sequences = tokenizer.texts_to_sequences(val_df['processed_tweet'])

MAX_LENGTH = max([len(x) for x in train_sequences])

# Pad the sequences
train_pad = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post')
test_pad = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post')
val_pad = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding='post')

# Extract the word index and vocab size
word_index = tokenizer.word_index
print("Unique tokens: " + str(len(word_index)))
vocab_size = len(word_index) + 1
print('Vocab size: ' + str(vocab_size))


Unique tokens: 7518
Vocab size: 7519


## Stances

In [5]:
def categorized_label(df, label_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df[label_name])
    categorical_labels = to_categorical(encoded_labels)
    return categorical_labels

In [6]:
train_labels = categorized_label(train_df, "Stance")
test_labels = categorized_label(test_df, "Stance")
val_labels = categorized_label(val_df, "Stance")

# [3] Create Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from keras import backend as K
from tensorflow.keras.optimizers.legacy import Adam


In [8]:
# Define the F1 score metric
def f1_score(y_true, y_pred):
    # Calculate Precision and Recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    # Calculate F1 score
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Set up call backs and learning rates

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Define the EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-5,
    verbose=1,
)

# Define the ModelCheckpoint callback
checkpoint_path = "best_model2.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,    # Only save a model if 'val_loss' has improved
    monitor='val_loss',     # Monitor 'val_loss' during training
    mode='min',             # The model is saved when 'val_loss' is minimized
    verbose=1)

# Combine all callbacks in a list
callbacks = [
    early_stopping,
    lr_scheduler,
    # model_checkpoint_callback
]

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

In [10]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=MAX_LENGTH))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))  # 3 classes: FAVOUR, AGAINST, NEITHER

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', f1_score])

2023-12-20 14:20:24.286967: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-12-20 14:20:24.286988: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-12-20 14:20:24.286994: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-12-20 14:20:24.287027: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-20 14:20:24.287043: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
history = model.fit(train_pad, train_labels, 
                    epochs=20, 
                    validation_data=(val_pad, val_labels), 
                    batch_size=32, 
                    callbacks=callbacks,
                    shuffle = True)

Epoch 1/20


2023-12-20 14:20:25.370567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


92/92 [==============================] - ETA: 0s - loss: 1.0274 - accuracy: 0.4653 - f1_score: 0.3068

2023-12-20 14:21:44.143376: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


92/92 [==============================] - 82s 870ms/step - loss: 1.0274 - accuracy: 0.4653 - f1_score: 0.3068 - val_loss: 1.0271 - val_accuracy: 0.5123 - val_f1_score: 0.0230 - lr: 0.0010
Epoch 2/20
92/92 [==============================] - 76s 827ms/step - loss: 0.5702 - accuracy: 0.7732 - f1_score: 0.7651 - val_loss: 1.0074 - val_accuracy: 0.5123 - val_f1_score: 0.4865 - lr: 0.0010
Epoch 3/20
92/92 [==============================] - 80s 872ms/step - loss: 0.2422 - accuracy: 0.9077 - f1_score: 0.9034 - val_loss: 1.0014 - val_accuracy: 0.5348 - val_f1_score: 0.5105 - lr: 0.0010
Epoch 4/20
92/92 [==============================] - 84s 916ms/step - loss: 0.1671 - accuracy: 0.9451 - f1_score: 0.9412 - val_loss: 1.0433 - val_accuracy: 0.5286 - val_f1_score: 0.5059 - lr: 0.0010
Epoch 5/20
92/92 [==============================] - 81s 881ms/step - loss: 0.1168 - accuracy: 0.9619 - f1_score: 0.9624 - val_loss: 1.3926 - val_accuracy: 0.5010 - val_f1_score: 0.4793 - lr: 0.0010
Epoch 6/20
92/92 [===